# Imports and Definitions of functions

In [68]:
import pandas as pd
from tqdm.notebook import tqdm as tqdm_notebook
import re
from collections import defaultdict

# Data reading

In [173]:
FILE_ORIGIN = 'dblpv13.json'
FILE_PREPROCESSED = 'data.json'

Используйте этот скрипт один раз, чтобы создать удобочитаемый json

In [88]:
# with open(FILE_ORIGIN, 'r') as f_origin, open(FILE_PREPROCESSED, 'w+') as f_preprocessed:
#     for line in tqdm_notebook(f_origin, total=450000000):
#         new_line = re.sub(r'NumberInt\((\d*)\)', r'\1', line)
#         if new_line in ('[\n', ']\n'):
#             new_line = ''
#         if new_line == '},\n':
#             new_line = '}\n'
#         else:
#             new_line = new_line[:-1]
#         f_preprocessed.write(new_line)

  0%|          | 0/450000000 [00:00<?, ?it/s]

In [91]:
CHUNKSIZE = 100000
MAX_OBSERVATIONS = 500000 #Set to 55000000 if want all rows
df = pd.DataFrame()
with pd.read_json(FILE_PREPROCESSED, orient='records', lines=True, chunksize=CHUNKSIZE, nrows=MAX_OBSERVATIONS) as reader:
    for chunk in tqdm_notebook(reader, total=MAX_OBSERVATIONS // CHUNKSIZE):
        df = pd.concat([df, pd.DataFrame(chunk)], ignore_index=True)

  0%|          | 0/5 [00:00<?, ?it/s]

# EDA

In [93]:
df.sample(5)

,_id,title,venue,year,keywords,n_citation,lang,authors,fos,page_start,page_end,volume,issue,issn,isbn,doi,pdf,url,abstract,references
150634,53e999bbb7602d970220016d,Report on customs treatment of computer software,"{'sid': 'journals/clsr', 't': 'J', 'raw': 'Com...",1986.0,[],0.0,en,"[{'_id': '53f39cacdabfae4b34a9e4d5', 'name': '...",NaN,6,7,1,5,Computer Law and Security Review: The Internat...,,10.1016/0267-3649(86)90004-X,None,[http://dx.doi.org/10.1016/0267-3649(86)90004-...,,NaN
47621,53e99822b7602d9702045753,The array features,"{'_id': '555036e57cea80f9541656e0', 'raw': 'Co...",1996.0,"[parallel computer, particular choice, array f...",0.0,en,"[{'name': 'John Reid', '_id': '5601b9f445cedb3...","[Array processing, Matrix calculus, Programmin...",323,331,18,4,,,10.1016/0920-5489(96)01010-0,None,[http://dx.doi.org/10.1016/0920-5489(96)01010-0],The array features were never controversial du...,NaN
83624,53e998a9b7602d97020e3836,A Generalized ABFT Technique Using a Fault Tol...,{'sid': 'JOURNAL OF CIRCUITS SYSTEMS AND COMPU...,2007.0,"[fault tolerance, back propagation, MLP networ...",6.0,en,"[{'_id': '53f42ec0dabfaedd74d48881', 'name': '...","[Convolutional code, Function approximation, C...",337,356,16,3,0218-1266,,10.1142/S0218126607003708,None,"[http://dx.doi.org/10.1142/S0218126607003708, ...","In this paper, a measure of sensitivity is def...","[53e9a562b7602d9702e85a4f, 53e9afccb7602d9703a..."
379969,53e99d0cb7602d97025c3fd4,Building-up of system level ESD modeling: Impa...,"{'_id': '539e72148314ff4cf4994374', 'type': 0,...",2013.0,[],28.0,en,"[{'_id': '53f4315cdabfaeb22f43aa1d', 'name': '...","[Capacitance, Decoupling (cosmology), Waveform...",221,228,53,2,0026-2714,,10.1016/j.microrel.2012.04.012,None,[http://dx.doi.org/10.1016/j.microrel.2012.04....,Résumé We present a methodology for precise me...,[558c0534e4b0cfb70a1b2ae6]
439477,53e99dfdb7602d97026bd339,Automatic parameter selection for denoising al...,"{'_id': '555036f27cea80f954168664', 'type': 0,...",2010.0,"[tid2008 database show, computationally heavy,...",334.0,en,"[{'_id': '542a7577dabfae646d564de0', 'name': '...","[Noise reduction, Singular value decomposition...",3116,3132,19,12,1941-0042,,10.1109/TIP.2010.2052820,None,"[http://dx.doi.org/10.1109/TIP.2010.2052820, h...",Across the field of inverse problems in image ...,"[558ad05f84ae84d265bfca97, 53e9ae0bb7602d97038..."


In [94]:
df.columns

Index(['_id', 'title', 'venue', 'year', 'keywords', 'n_citation', 'lang',
       'authors', 'fos', 'page_start', 'page_end', 'volume', 'issue', 'issn',
       'isbn', 'doi', 'pdf', 'url', 'abstract', 'references'],
      dtype='object')

In [110]:
df._id.nunique()

500000

In [128]:
df['venue_keys'] = df.venue.apply(lambda x: list(dict(x).keys()) if isinstance(x, dict) else [])

In [134]:
venue_keys = defaultdict(int)
for val in df.venue_keys:
    for key in val:
        venue_keys[key] += 1

In [139]:
print("Статистика по venue:")
for key, occurance in venue_keys.items():
    print(f'{key} встречался {occurance} раз')

Статистика по venue:
type встречался 425136 раз
_id встречался 426541 раз
name_d встречался 205684 раз
raw встречался 495618 раз
raw_zh встречался 59240 раз
publisher встречался 47491 раз
sid встречался 54666 раз
issn встречался 50477 раз
t встречался 54580 раз
online_issn встречался 16790 раз
name_s встречался 177 раз
name встречался 657 раз
src встречался 16 раз


In [141]:
keywords = defaultdict(int)
for val in df.keywords:
    if not isinstance(val, list):
        keywords['nothing'] += 1
    else:
        for key in val:
            keywords[key] += 1

In [151]:
print("Статистика по top keywords:")
for key in sorted(keywords.keys(), key=lambda x: keywords[x], reverse=True)[:10]:
    print(f'{key} встречался {keywords[key]} раз')

Статистика по top keywords:
data mining встречался 8094 раз
computer science встречался 6149 раз
real time встречался 5576 раз
computational complexity встречался 5571 раз
feature extraction встречался 5351 раз
internet встречался 5202 раз
satisfiability встречался 5121 раз
neural network встречался 4272 раз
quality of service встречался 4127 раз
protocols встречался 4045 раз


In [152]:
print(keywords['nothing'])

147


In [171]:
authors_keys = defaultdict(int)
for val in df.authors:
    if not isinstance(val, list):
        authors_keys['noone'] += 1
    else:
        for dic in val:
            for key in dict(dic):
                authors_keys[key] += 1

In [172]:
print("Статистика по authors:")
for key, occurance in authors_keys.items():
    print(f'{key} встречался {occurance} раз')

Статистика по authors:
noone встречался 3129 раз
_id встречался 1293241 раз
name встречался 1318668 раз
org встречался 683420 раз
gid встречался 509455 раз
oid встречался 25175 раз
orgid встречался 562592 раз
sid встречался 37961 раз
email встречался 190274 раз
orgs встречался 314092 раз
orcid встречался 47590 раз
bio встречался 19692 раз
name_zh встречался 411 раз
org_zh встречался 408 раз
oid_zh встречался 26 раз
orgs_zh встречался 71 раз


In [157]:
keywords_fos = defaultdict(int)
for val in df.fos:
    if not isinstance(val, list):
        keywords_fos['nothing'] += 1
    else:
        for key in val:
            keywords_fos[key] += 1

In [158]:
print("Статистика по top keywords_fos:")
for key in sorted(keywords_fos.keys(), key=lambda x: keywords_fos[x], reverse=True)[:10]:
    print(f'{key} встречался {keywords_fos[key]} раз')

Статистика по top keywords_fos:
Computer science встречался 280921 раз
Mathematics встречался 98092 раз
Artificial intelligence встречался 81806 раз
Algorithm встречался 38111 раз
nothing встречался 37151 раз
Engineering встречался 28961 раз
Discrete mathematics встречался 28528 раз
Computer vision встречался 27672 раз
Computer network встречался 27395 раз
Combinatorics встречался 23680 раз


In [159]:
df.issn.nunique()

7780

In [163]:
df.issn.drop_duplicates().shape

(7781,)

In [164]:
df.isbn.drop_duplicates().shape

(14552,)

In [165]:
df.isbn.nunique()

14551

In [166]:
df.doi.nunique()

398433

In [167]:
df.doi.drop_duplicates().shape

(398434,)

In [168]:
df.pdf.nunique()

141884

In [169]:
df.pdf.drop_duplicates().shape

(141886,)

In [170]:
df.pdf.value_counts()

                                                                                      15324
//static.aminer.org/pdf/PDF/000/831/798/book_review.pdf                                  10
//static.aminer.org/pdf/PDF/000/737/991/editorial.pdf                                     8
//static.aminer.org/pdf/PDF/000/840/383/book_reviews.pdf                                  6
//static.aminer.org/pdf/PDF/000/889/226/preface.pdf                                       5
                                                                                      ...  
//static.aminer.org/pdf/PDF/000/560/826/coaching_advice_and_adaptation.pdf                1
https://static.aminer.cn/upload/pdf/146/831/774/53e99a0fb7602d9702262d62_0.pdf            1
//static.aminer.org/pdf/PDF/000/091/394/culture_and_collaborative_technologies.pdf        1
https://static.aminer.cn/upload/pdf/138/831/774/53e99a0fb7602d9702262d5a_0.pdf            1
https://static.aminer.cn/upload/pdf/779/311/1355/53e99eedb7602d97027b67fb_1.pdf 

# Краткая статистика по полям

<pre>
_id - string, уникальное значение статьи
title - string, название статьи
venue - dict, информация о том, где была опубликована
    - _id - id места публикации 
    - name_d - название места публикации
    - дальше не понятно, возможные ключи можете увидеть выше
year - int, год публикации
keywords - list, ключивые слова статей
n_citation - int, количество цитирований
lang - string, язык
authors - list[dict], информация об авторах
    - смотреть статистику выше
fos - list, очень похоже на keywords, но пока не могу точно характеризовать в чем разница
page_start - int, номер страницы в публикации начиная с которой идет статья
page_end - int, номер страницы в публикации на которой заканчивается статья
volume - int, сколько лет статья уже публикуется
issue - int, сколько раз была опубликована в течении данного year
issn - string, уникальный номер, но есть не у всех
isbn - string, тоже уникальный номер, но есть не у всех
doi - string, тоже уникальный номер и тоже есть не у всех
pdf - string, выглядит как ссылка на статью в aminer
url - string, ссылка с использованием doi
abstract - string, краткое описание статьи
references - list[_id], id статей, которые цитировались в данной статье]
</pre>